In [1]:
! pip install pandas

In [2]:
import pandas as pd
import glob as gb
from pprint import pprint
from io import StringIO
from datetime import datetime

In [3]:
DIR = '**'
logs_paths = gb.glob(DIR, recursive=True)
logs_paths = [p for p in logs_paths if ".log" in p]
logs_paths = [p for p in logs_paths if "dag_id=app_template" in p]
logs_paths[:10]

['tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=loaders.load_1/attempt=1.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=transformers.transform_3/attempt=1.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=loaders.load_4/attempt=2.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=loaders.load_4/attempt=4.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=loaders.load_4/attempt=3.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=loaders.load_3/attempt=1.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=transformers.transform_2/attempt=1.log',
 'tmp/dag_id=app_template/run_id=manual__096dcecb-3f0c-48ce-b18a-750bd40f9488/task_id=transformers.transform_1/attempt=1.log',
 'tmp/dag_id=app_template/run_id=m

In [4]:
df_logs = pd.DataFrame()
for path in logs_paths:

    with open(path, 'r') as f:
        lines = f.readlines()
        lines = [l for l in lines if '{helpers.py:111}' in l]
        lines = ''.join(lines)
    try:
        df = pd.read_csv(StringIO(lines), sep=' ', header=None)
    except:
        continue

    df = df[df[1]=='{helpers.py:111}']
    col_names = {
        0: 'timestamp',
        4: 'tasks',
        5: 'batch_id',
        6: 'processed_%',
        7: 'seconds'
    }
    df = df[col_names.keys()]
    df.rename(columns=col_names, inplace=True)
    task_type = path.split('/')[4].split('=')[1].split('.')[0]
    df['task_type'] = task_type
    run_id = path.split('/')[2].split('=')[1].split('__')[1]
    df['run_id'] = run_id
    df_logs = pd.concat([df_logs, df], ignore_index=True)
df_logs.reset_index(drop=True, inplace=True)

df_logs['seconds'] = pd.to_numeric(df_logs['seconds'], errors='coerce')
df_logs['processed_%'] = pd.to_numeric(df_logs['processed_%'], errors='coerce')
df_logs['timestamp'] = \
    pd.to_datetime(df_logs['timestamp'].str.strip('[]'), errors='coerce')

display(df_logs.shape)
df_logs.sample(10)

(260767, 7)

,timestamp,tasks,batch_id,processed_%,seconds,task_type,run_id
205553,2025-08-27 10:05:10.612000+00:00,1-1-1,102,81.60,2.021,1,b1ddb338-6985-4034-a691-25c74c1e89a3
212876,2025-08-27 09:21:18.622000+00:00,4-4-2,9,29.03,2.891,1,eab7945d-3b15-4e65-939c-22b4315ccc6b
104722,2025-08-27 09:08:32.442000+00:00,4-4-2,3,12.00,1.850,1,e8b96ee3-8144-4f63-80a7-4e2ae3ff654e
35969,2025-08-27 08:33:49.460000+00:00,4-2-4,20,64.52,2.443,1,35c5f91b-cd04-4d31-8f04-34dcefc9532f
144938,2025-08-27 03:08:59.403000+00:00,2-1-2,38,61.29,2.183,1,d9932300-777f-4bec-86d8-45c248a396dd
46171,2025-08-27 02:29:49.050000+00:00,1-5-3,9,21.95,2.187,3,f55d54b8-fa67-4813-b5c3-2403c83a70b7
131099,2025-08-27 07:43:42.712000+00:00,4-1-3,123,98.40,3.501,1,565c9750-d5a6-4896-829f-12c82eada11e
18783,2025-08-27 09:19:05.226000+00:00,4-3-3,23,74.19,1.963,1,02db64b0-d00c-41f5-b1a1-ba9e8f95d846
246996,2025-08-27 00:54:07.714000+00:00,1-1-5,1,4.00,2.708,1,4905e34e-8d99-4a32-8b15-3c2816cfee2a
41497,2025-08-27 04:17:17.092000+00:00,2-3-4,11,35.48,2.584,1,34f16280-8ec3-4b7e-b8d5-01791dbab1e7


In [10]:
task_counts = df_logs["tasks"].value_counts().reset_index()
task_counts.columns = ["tasks", "count"]
df_logs_filtered = df_logs.groupby("tasks").head(3125)
df_logs_filtered.shape

(244214, 7)

In [9]:
25*125

3125

In [6]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#df_logs.to_csv(f"tmp/logs_{timestamp}.csv", index=False)